# Using environments with Azure ML


## Introduction

Azure ML environments are an encapsulation of the environment where your machine learning training happens. They define Python packages, environment variables, Docker settings and other attributes in declarative fashion. Environments are versioned: you can update them and retrieve old versions to revisit and review your work.

Environments allow you to:
* Encapsulate dependencies of your training process, such as Python packages and their versions.
* Reproduce the Python environment on your local computer in a remote run on VM or ML Compute cluster
* Reproduce your experimentation environment in production setting.
* Revisit and audit the environment in which an existing model was trained.

Environment, compute target and training script together form run configuration: the full specification of training run.

## Setup

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't.

First, let's validate Azure ML SDK version and connect to workspace.

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
!pip list | wc -l

481


In [3]:
import azureml.core
print(azureml.core.VERSION)

1.30.0


In [4]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/azuremlworkshop2021-rg/providers/Microsoft.MachineLearningServices/workspaces/azuremlworkshop2021',
 'name': 'azuremlworkshop2021',
 'identity': {'principal_id': '40b6de37-60f7-41d1-89a8-a193fb743fb5',
  'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'e0149dc7-80d7-4e18-a7a1-eb801f23ae2b',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': '',
 'friendlyName': 'azuremlworkshop2021',
 'creationTime': '2021-06-23T08:59:19.2642912+00:00',
 'containerRegistry': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop2021-rg/providers/microsoft.containerregistry/registries/azuremlworkshop2021',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop2021

## Use curated environments

Curated environments are provided by Azure Machine Learning and are available in your workspace by default. They contain collections of Python packages and settings to help you get started different machine learning frameworks. 

  * The __AzureML-Minimal__ environment contains a minimal set of packages to enable run tracking and asset uploading. You can use it as a starting point for your own environment.
  * The __AzureML-Tutorial__ environment contains common data science packages, such as Scikit-Learn, Pandas and Matplotlib, and larger set of azureml-sdk packages.
 
Curated environments are backed by cached Docker images, reducing the run preparation cost.
 
You can get a curated environment using

In [5]:
from azureml.core import Environment

curated_env = Environment.get(workspace=ws, name="AzureML-Minimal")

To list curated environments, use following code.

**Note**: The name prefixes _AzureML_ and _Microsoft_ are reserved for curated environments. Do not use them for your own environments

In [6]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-tensorflow-1.15-ubuntu18.04-py37-cpu-inference
packages channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults==1.30.0
name: project_environment

Name AzureML-Triton
packages channels:
- conda-forge
dependencies:
- python=3.7.9
- pip:
  - azureml-core==1.30.0
  - azureml-defaults[async]
  - azureml-contrib-services==1.30.0
  - numpy
  - inference-schema[numpy-support]
  - grpcio-tools
  - geventhttpclient
  - https://developer.download.nvidia.com/compute/redist/tritonclient/tritonclient-2.7.0-py3-none-manylinux1_x86_64.whl
name: azureml_08e35304262dcb8bb02bcca41661f10b

Name AzureML-PyTorch-1.6-CPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.30.0
  - azureml-defaults==1.30.0
  - azureml-telemetry==1.30.0
  - azureml-train-restclients-hyperdrive==1.30.0
  - azureml-train-core==1.30.0
  - cmake==3.18.2
  - torch==1.6.0
  - torchvision==0.5.0
  - mkl==2018.0.3
  - horovod==0.21.3
  

## Create your own environment

You can create an environment by instantiating ```Environment``` object and then setting its attributes: set of Python packages, environment variables and others.

### Add Python packages

The recommended way is to specify Conda packages, as they typically come with complete set of pre-built binaries.

In [7]:
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

You can also add pip packages, and specify the version of package

In [8]:
conda_dep.add_pip_package("pillow==5.4.1")
myenv.python.conda_dependencies=conda_dep

### Specify environment variables

You can add environment variables to your environment. These then become available using ```os.environ.get``` in your training script.

In [9]:
myenv.environment_variables = {"MESSAGE":"Hello from Azure Machine Learning"}

## Register environment

You can manage environments by registering them. This allows you to track their versions, and reuse them in future runs. For example, once you've constructed an environment that meets your requirements, you can register it and use it in other experiments so as to standardize your workflow.

If you register the environment with same name, the version number is increased by one. Note that Azure ML keeps track of differences between the version, so if you re-register an identical version, the version number is not increased.

In [14]:
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210513.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "MESSAGE": "Hello from Azure Machine Learning"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-for

## List and get existing environments

Your workspace contains a dictionary of registered environments. You can then use ```Environment.get``` to retrieve a specific environment with specific version.

In [15]:
for name,env in ws.environments.items():
    print("Name {} \t version {}".format(name,env.version))

restored_environment = Environment.get(workspace=ws,name="myenvserge",version="1")

print("Attributes of restored environment")
restored_environment

Name tutorial-env 	 version 2
Name myenv 	 version 1
Name AzureML-tensorflow-1.15-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-Triton 	 version 16
Name AzureML-PyTorch-1.6-CPU 	 version 18
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11.0.3-gpu-inference 	 version 2
Name AzureML-pytorch-1.7-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-minimal-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-DeepSpeed-0.3-GPU 	 version 10
Name AzureML-xgboost-0.9-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-pytorch-1.6-ubuntu18.04-py37-cpu-inference 	 version 2
Name AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu 	 version 3
Name AzureML-TensorFlow-2.3-CPU 	 version 14
Name AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu 	 version 3
Name AzureML-

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definition exists for environment myenvserge version 1",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "4a29c99885c6f14db3e26eec23dd1381",
    "request": "0ca9a52ef9b6ee45"
  },
  "environment": "westeurope",
  "location": "westeurope",
  "time": "2021-06-23T09:34:40.5936182+00:00",
  "componentName": "environment-management"
}

## Other ways to create environments

### From existing Conda environment

You can create an environment from existing conda environment. This make it easy to reuse your local interactive environment in Azure ML remote runs. For example, if you've created conda environment using
```
conda create -n mycondaenv
```
you can create Azure ML environment out of that conda environment using
```
myenv = Environment.from_existing_conda_environment(name="myenv",conda_environment_name="mycondaenv")
```

### From conda or pip files

You can create environments from conda specification or pip requirements files using
```
myenv = Environment.from_conda_specification(name="myenv", file_path="path-to-conda-specification-file")

myenv = Environment.from_pip_requirements(name="myenv", file_path="path-to-pip-requirements-file")
```


## Using environments for inferencing

You can re-use the training environment when you deploy your model as a web service, by specifying inferencing stack version, and adding then environment to ```InferenceConfig```.

```
from azureml.core.model import InferenceConfig

myenv.inferencing_stack_version = "latest"

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
```

See [Register Model and deploy as Webservice Notebook](../../deployment/deploy-to-cloud/model-register-and-deploy.ipynb) for an end-to-end example of web service deployment.